# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from math import nan
from scipy import stats

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata

,eeg_file,ParticipantID,test,sex,age_months,age_months_days,dyslexic_parent,Group_AccToParents,eeg_path,path_epoch,epoch_file
0,101a,101,a,m,20,20;22,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,101a_epo.fif
1,101b,101,b,m,23,23;16,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,101b_epo.fif
2,102a,102,a,f,20,20;27,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,102a_epo.fif
3,102b,102,b,f,23,23;16,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,102b_epo.fif
4,103a,103,a,f,20,20;23,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,103a_epo.fif
...,...,...,...,...,...,...,...,...,...,...,...
57,134a,134,a,m,19,19;20,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,134a_epo.fif
58,134b,134,b,m,23,23;7,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,134b_epo.fif
59,135a,135,a,f,18,18;21,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,135a_epo.fif
60,135b,135,b,f,22,22;8,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,135b_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.3 to 0.7.

In [4]:
def read_filtered_data(metadata):
    epochs = []
    for index, file in metadata.iterrows():
        print(f"Checking out file: {file['epoch_file']}")
        path = os.path.join(file['path_epoch'], file['epoch_file'])
        epoch = mne.read_epochs(path, preload=False)
        epochs.append(epoch)
    return epochs

In [5]:
epochs = read_filtered_data(metadata)

Checking out file: 101a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\101a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2266 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 101b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\101b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 102a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\102a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2389 matching events found
No baseline correction applied
0 p

Checking out file: 115a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\115a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
1220 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 115b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\115b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
1220 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 116a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\116a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2266 matching events found
No baseline correction applied
0 p

Checking out file: 129b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\129b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2302 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 130a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\130a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2389 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 130b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\130b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 p

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [124]:
from scipy.stats import skew
# define the events for standard and deviant
standard_events = [2,5,8,11]
deviant_events = [3,6,9,12]

def input_mmr_prep(metadata, epochs): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file", "paradigm", "channel", "mean"])

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        
        # loop over every paradigm per participant 
        for j in standard_events: 
            paradigm = j
            # select the standard and deviant for a specific sequence and calculate the evoked response
            std_evoked = epochs[i][j].average() 
            dev_evoked = epochs[i][j+1].average()
            
            # calculate the mismatch response between standard and deviant evoked
            evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
            
            # get a list of all channels
            chnames_list = evoked_diff.info['ch_names']
            
            # compute for every channel the distance mean of the mismatch line
            for channel in chnames_list: 
                chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
                roi_dict = dict(left_ROI=chnames) # unfortunately combine_channels only takes a dictionary as input
                roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
                mmr = roi_evoked.to_data_frame()
                mmr_avg = mmr['left_ROI'].mean()
                mmr_std = mmr['left_ROI'].std()
                mmr_skew = mmr['left_ROI'].skew()
                
                df = df.append({'eeg_file': metadata['eeg_file'][i], 'paradigm' : paradigm, 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew}, ignore_index=True) 
    return df

In [ ]:
df = input_mmr_prep(metadata, epochs)

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

...

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-124-0cfe375b579a>:30: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000288F78A8C10>, axis=0)".
  r

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [8]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['paradigm','channel'])
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]
df.reset_index(inplace=True)

In [9]:
df.columns = df.columns.str[5:]
df = df.rename(columns={'ile':'eeg_file'})

In [10]:
df

,eeg_file,2_Fp1,2_AF3,2_F7,2_F3,2_FC1,2_FC5,2_T7,2_C3,2_CP1,...,11_C4,11_T8,11_FC6,11_FC2,11_F4,11_F8,11_AF4,11_Fp2,11_Fz,11_Cz
0,101a,3.981499,11.639081,-18.811622,18.343454,10.023632,-4.024143,-13.999364,-17.671889,-29.638338,...,-5.419709,-20.527842,-7.739499,-24.687588,-40.535996,-6.189181,-28.356975,-19.061485,-40.427852,-43.257143
1,101b,-9.964857,-7.810340,19.733811,3.661548,14.292444,32.492531,34.867429,41.204902,33.612812,...,-3.435515,1.738558,8.090447,13.989412,8.420473,23.518492,50.627080,33.244638,28.191358,40.034364
2,102a,2.390371,-17.843979,-3.338771,7.536740,-10.998419,18.220947,16.625307,-4.156407,-17.935160,...,26.739956,3.039871,15.349378,31.937126,41.415109,14.678230,22.992495,65.963604,31.134918,18.108797
3,102b,7.074118,23.187111,-9.459183,25.835532,18.345802,34.434200,34.885866,22.527005,37.353435,...,15.112951,36.641937,37.896288,19.602109,14.555350,41.145307,24.244979,58.820555,11.613820,9.621704
4,103a,-18.065883,-17.785199,-12.858838,-8.709094,9.764843,-1.953743,-2.150446,11.090374,0.289517,...,-39.471842,-45.274814,-41.629402,-50.573104,-69.076174,-59.793993,-69.507558,-66.936390,-66.624167,-50.016909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,134a,25.305783,39.678430,29.960605,22.644373,2.972337,40.967793,15.140849,-5.808729,2.409057,...,-11.897622,11.621626,1.961869,-10.079748,-10.880721,-3.558657,18.596759,21.565529,18.908299,-5.037781
58,134b,-9.407622,-9.174790,-29.506209,-12.091466,-3.002780,-12.201141,-28.593497,9.314769,-14.475088,...,26.460729,69.404226,24.898261,-6.394242,8.813215,36.010730,3.894987,12.425814,-100.931252,-3.595413
59,135a,-24.276921,-15.623758,-40.422530,-1.687877,11.960549,-12.416261,1.713572,26.404282,4.149861,...,-3.751271,-18.935111,-7.520779,7.278066,-8.365071,50.907542,3.610985,-4.971196,1.646921,25.449629
60,135b,-18.316388,-1.091161,-33.190223,-7.661494,19.655141,1.925744,8.144818,4.500359,-0.237321,...,-31.523276,-57.036920,-40.660919,-23.062408,-16.706054,-42.632576,-9.729258,-2.055501,-14.779368,-37.734673


## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [11]:
df = pd.merge(df, metadata, on='eeg_file')

In [12]:
df.columns

Index(['eeg_file', '2_Fp1', '2_AF3', '2_F7', '2_F3', '2_FC1', '2_FC5', '2_T7',
       '2_C3', '2_CP1',
       ...
       'ParticipantID', 'test', 'sex', 'age_months', 'age_months_days',
       'dyslexic_parent', 'Group_AccToParents', 'eeg_path', 'path_epoch',
       'epoch_file'],
      dtype='object', length=139)

Drop some unnecessary columns. 

In [13]:
df = df.drop(['eeg_file','age_months_days',
       'dyslexic_parent', 'eeg_path', 'path_epoch',
       'epoch_file'], axis =1)

Every participant has now two rows, one for each test. We now want a single row for each participant, since the participant already has experience with the test and therefore the test at time b might differ from the test atime a. To avoid bias in the data, both tests will be appended to the participant into one row. 

In [14]:
a = df.loc[df['test'] == 'a']
b = df.loc[df['test'] == 'b']
a = a.drop(['test'],axis=1)
b = b.drop(['test','sex','age_months','Group_AccToParents'], axis=1)

In [15]:
df = pd.merge(a, b, on='ParticipantID',suffixes=("_a", "_b"))

In [16]:
df['sex'] = np.where((df['sex']=='m'), 1,0)
df['Group_AccToParents'] = np.where((df['Group_AccToParents']=='At risk'), 1,0)

In [17]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

In [ ]:
hist = df.hist(column='2_AF3_a',by=df['Group_AccToParents'],color='#12ABDB',sharex=True, figsize=(8,5), xlabelsize=11, ylabelsize=11,bins=40)

In [19]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

In [20]:
df

,Group_AccToParents,2_Fp1_a,2_AF3_a,2_F7_a,2_F3_a,2_FC1_a,2_FC5_a,2_T7_a,2_C3_a,2_CP1_a,...,11_C4_b,11_T8_b,11_FC6_b,11_FC2_b,11_F4_b,11_F8_b,11_AF4_b,11_Fp2_b,11_Fz_b,11_Cz_b
0,1,3.981499,11.639081,-18.811622,18.343454,10.023632,-4.024143,-13.999364,-17.671889,-29.638338,...,-3.435515,1.738558,8.090447,13.989412,8.420473,23.518492,50.627080,33.244638,28.191358,40.034364
1,0,2.390371,-17.843979,-3.338771,7.536740,-10.998419,18.220947,16.625307,-4.156407,-17.935160,...,15.112951,36.641937,37.896288,19.602109,14.555350,41.145307,24.244979,58.820555,11.613820,9.621704
2,1,-18.065883,-17.785199,-12.858838,-8.709094,9.764843,-1.953743,-2.150446,11.090374,0.289517,...,24.699454,32.055830,-4.361878,-5.785746,12.088077,9.451175,6.296128,-12.169311,0.450006,6.585673
3,1,52.196651,51.457712,22.673943,32.572107,5.180163,18.782386,20.845702,7.723524,5.208093,...,52.046497,54.323672,58.501746,56.353772,49.522784,71.824112,42.726944,74.669478,39.794922,62.994571
4,1,-841.737108,-801.240011,-879.743229,-923.354318,-801.476465,-847.688256,-737.339387,-853.147914,-732.422316,...,-41.609304,-13.818254,-30.328267,-57.140936,-32.112574,-38.012191,-43.552506,-43.767460,-61.546375,-55.303023
5,1,1.179806,6.366317,-22.585827,-16.469787,-26.659719,-20.820073,-44.714848,-5.861646,-4.621675,...,-9.086049,-28.215221,23.216179,22.833120,18.658916,-4.978932,16.195296,15.155379,19.184594,8.517291
6,1,-104.473044,-114.661477,-115.992143,-104.786412,-97.066618,-104.369716,-116.949484,-105.987364,-99.901829,...,11.753093,17.640083,-9.471046,4.256348,3.366350,-6.142375,-9.982942,-23.373584,1.966171,3.573784
7,1,-36.590509,-35.888334,-32.303728,-41.009655,-39.813567,-35.966233,-32.699715,-45.066325,-43.378386,...,-12.600397,-3.867548,-47.104975,-9.607376,-15.133582,-21.744324,-5.319063,-5.319112,-7.363014,-18.200347
8,1,-13.833605,-3.938048,11.836307,-27.299734,-8.398938,-128.276705,25.861200,-4.607624,-26.808643,...,-15.484524,-0.724520,2.189864,2.951773,3.218586,21.250784,-9.144113,-8.693288,5.496910,-12.589195
9,0,32.232969,33.826705,38.582741,10.956048,42.392605,35.946838,31.716506,53.214490,52.874939,...,-21.800947,6.808585,-8.667056,-23.001667,-9.690261,18.572201,-42.575242,-48.704422,-22.459858,-52.576538


In [21]:
df[(np.abs(stats.zscore(df)) < 5).all(axis=1)] # z score 

,Group_AccToParents,2_Fp1_a,2_AF3_a,2_F7_a,2_F3_a,2_FC1_a,2_FC5_a,2_T7_a,2_C3_a,2_CP1_a,...,11_C4_b,11_T8_b,11_FC6_b,11_FC2_b,11_F4_b,11_F8_b,11_AF4_b,11_Fp2_b,11_Fz_b,11_Cz_b
0,1,3.981499,11.639081,-18.811622,18.343454,10.023632,-4.024143,-13.999364,-17.671889,-29.638338,...,-3.435515,1.738558,8.090447,13.989412,8.420473,23.518492,50.627080,33.244638,28.191358,40.034364
1,0,2.390371,-17.843979,-3.338771,7.536740,-10.998419,18.220947,16.625307,-4.156407,-17.935160,...,15.112951,36.641937,37.896288,19.602109,14.555350,41.145307,24.244979,58.820555,11.613820,9.621704
3,1,52.196651,51.457712,22.673943,32.572107,5.180163,18.782386,20.845702,7.723524,5.208093,...,52.046497,54.323672,58.501746,56.353772,49.522784,71.824112,42.726944,74.669478,39.794922,62.994571
5,1,1.179806,6.366317,-22.585827,-16.469787,-26.659719,-20.820073,-44.714848,-5.861646,-4.621675,...,-9.086049,-28.215221,23.216179,22.833120,18.658916,-4.978932,16.195296,15.155379,19.184594,8.517291
6,1,-104.473044,-114.661477,-115.992143,-104.786412,-97.066618,-104.369716,-116.949484,-105.987364,-99.901829,...,11.753093,17.640083,-9.471046,4.256348,3.366350,-6.142375,-9.982942,-23.373584,1.966171,3.573784
7,1,-36.590509,-35.888334,-32.303728,-41.009655,-39.813567,-35.966233,-32.699715,-45.066325,-43.378386,...,-12.600397,-3.867548,-47.104975,-9.607376,-15.133582,-21.744324,-5.319063,-5.319112,-7.363014,-18.200347
8,1,-13.833605,-3.938048,11.836307,-27.299734,-8.398938,-128.276705,25.861200,-4.607624,-26.808643,...,-15.484524,-0.724520,2.189864,2.951773,3.218586,21.250784,-9.144113,-8.693288,5.496910,-12.589195
10,1,29.180499,51.305511,33.684065,77.669493,33.498453,42.805475,17.245443,28.664506,9.766044,...,-26.917469,-7.233477,-8.595354,-33.983719,-36.807665,-30.939699,-41.274768,-42.812670,-48.965761,-35.948495
11,1,22.242199,25.520269,39.872521,19.348655,162.051226,22.990998,-6.394020,15.786654,29.528279,...,-4.763531,16.496005,6.222413,-2.886372,3.912438,7.668884,5.349972,17.297770,-7.735792,3.123703
13,0,38.784592,29.259179,26.296711,40.195044,44.741321,13.358658,22.685613,22.591467,38.658071,...,9.454800,2.076840,-1.722922,-0.105210,-33.125714,7.837773,-29.191126,-28.807280,-22.898105,17.743030


In [87]:
avgdf = df.groupby('Group_AccToParents').mean()

In [88]:
avgdf['mean'] = avgdf.mean(axis=1)

In [89]:
avgdf

,2_Fp1_a,2_AF3_a,2_F7_a,2_F3_a,2_FC1_a,2_FC5_a,2_T7_a,2_C3_a,2_CP1_a,2_CP5_a,...,11_T8_b,11_FC6_b,11_FC2_b,11_F4_b,11_F8_b,11_AF4_b,11_Fp2_b,11_Fz_b,11_Cz_b,mean
Group_AccToParents,,,,,,,,,,,,,,,,,,,,,
0,-2.475829,-3.042791,-3.168054,-0.651654,-0.256922,-122.646546,2.472113,11.150288,-2.911240,-0.393297,...,9.136860,2.702087,1.496306,-0.847739,8.208100,-4.082869,2.336325,-10.427652,-1.481018,0.741959
1,-60.513481,-56.094856,-63.368118,-64.875916,-50.465576,-70.598089,-61.072619,-67.699920,-60.609931,-59.855244,...,10.339962,8.087704,6.070160,11.508628,12.612424,7.556610,5.127068,3.534297,3.063697,-10.721792
